<h1 style="text-align: center;">Handling netcdf files using xarray</h1>

##### Overview of all available data visualization packages in python   https://pyviz.org/

### Why xarray? <br> (xarray main characteristics)

#### Database-like alignment based on coordinate labels that smoothly handles missing values: x, y = xr.align(x, y, join='outer')

This feature of xarray allows you to align two or more xarray objects (such as DataArray or Dataset) based on coordinate labels, even in the presence of missing values.

In practice, the function xr.align(x, y, join='outer') takes two xarray objects (x and y) and aligns them so that they have the same coordinates. The join='outer' option creates a union of the coordinates of x and y, inserting NaN where data is missing in one of the two objects.

In [ ]:
import pandas as pd
import xarray as xr
import numpy as np

In [ ]:
da1 = xr.DataArray([1, 2, 3], coords=[('x', [10, 20, 30])])
da2 = xr.DataArray([4, 5], coords=[('x', [20, 40])])


In [ ]:
da1 + da2

In [ ]:
da1,da2 = xr.align(da1, da2, join='outer')

In [ ]:
da1

In [ ]:
da2

In [ ]:
da1.fillna(0) + da2.fillna(0)

#### Apply operations over dimensions by name: x.sum('time')

In [ ]:
# Create a DataArray
data = xr.DataArray(
    np.random.rand(10, 5, 5),
    dims=("time", "lat", "lon"),
    coords={
        "time": pd.date_range("2023-01-01", periods=10),
        "lat": np.arange(5),
        "lon": np.arange(5),
    },
)
print(data)


In [ ]:
# Sum along "time" dimension
data_sum = data.sum(dim="time")

print(data_sum)

#### Select values by label (or logical location) instead of integer location: x.loc['2014-01-01'] or x.sel(time='2014-01-01')

In [ ]:

# Crea un DataArray di esempio
data = xr.DataArray(
    [100, 200, 300, 400, 500],
    dims="time",
    coords={"time": pd.date_range("2014-01-01", periods=5)},
)

# Seleziona il valore corrispondente alla data '2014-01-03'
value_03 = data.loc['2014-01-03']

# Seleziona il valore in cui la coordinata 'time' è uguale a '2014-01-05'
value_05 = data.sel(time='2014-01-05')

In [ ]:
print(f"value_03: {value_03}")
print(f"value_05: {value_05}")

#### Mathematical operations (e.g., x - y) vectorize across multiple dimensions (array broadcasting) based on dimension names, not shape

In this example, arr1 has dimensions lat and lon, while arr2 has only the lon dimension. Despite the different shapes, xarray can perform the subtraction correctly because it aligns the two arrays based on the name of the lon dimension. If we had used NumPy, we would have had to manually reshape arr2 to match the shape of arr1 before performing the subtraction. Xarray simplifies this process by automatically aligning the arrays based on dimension names.

In [ ]:
# Crea due DataArray con dimensioni diverse
arr1 = xr.DataArray([[1, 2], [3, 4]], dims=["lat", "lon"])
arr2 = xr.DataArray([5, 6], dims=["lon"])

# Esegui una sottrazione
result = arr1 - arr2

# Stampa il risultato
print(result)

#### Easily use the split-apply-combine paradigm with groupby: x.groupby('time.dayofyear').mean().

In this example, data.mean(dim="tempo") calculates the average of the values along the "tempo" dimension, while data.sum(dim="latitudine") calculates the sum along the "latitudine" dimension. Xarray identifies the dimensions by name, simplifying operations and making the code more readable.

In [ ]:

# Crea una serie temporale multi-anno
time = pd.date_range("2015-01-01", "2020-12-31", freq="D")

# Genera temperature sintetiche con una componente stagionale e casuale
np.random.seed(42)
temperature = 10 + 5 * np.sin(2 * np.pi * time.dayofyear / 365) + np.random.normal(scale=2, size=len(time))

# Crea un oggetto xarray
ds = xr.DataArray(
    data=temperature,
    coords={"time": time},
    dims=["time"],
    name="temperature"
)
print(ds)

In [ ]:
# Calcola la media per ogni giorno dell'anno
daily_mean = ds.groupby("time.dayofyear").mean()

print(daily_mean)

In [ ]:
import matplotlib.pyplot as plt
# Plot delle medie giornaliere
plt.figure(figsize=(10, 5))
plt.plot(daily_mean.dayofyear, daily_mean, label="Climatologia media")
plt.title("Climatologia media giornaliera (2015-2020)")
plt.xlabel("Giorno dell'anno")
plt.ylabel("Temperatura (°C)")
plt.grid()
plt.legend()
plt.show()

In [ ]:
monthly_mean = ds.groupby("time.month").mean()
print(monthly_mean)

In [ ]:
# Plot delle medie giornaliere
plt.figure(figsize=(10, 5))
plt.plot(monthly_mean.month, monthly_mean, label="Climatologia media")
plt.title("Climatologia media mensile (2015-2020)")
plt.xlabel("Mese dell'anno")
plt.ylabel("Temperatura (°C)")
plt.grid()
plt.legend()
plt.show()

#### Keep track of arbitrary metadata in the form of a Python dictionary: x.attrs

The x.attrs attribute in xarray is a Python dictionary that allows arbitrary metadata to be attached to a DataArray or Dataset object. It is useful for providing additional information about the data, such as its source, units of measurement, a description, or any other relevant annotations.

In [ ]:
# Creiamo un semplice DataArray con dati di temperatura
time = pd.date_range("2023-01-01", periods=10, freq="D")
temperature = np.random.uniform(15, 25, size=len(time))

temperature_da = xr.DataArray(
    data=temperature,
    coords={"time": time},
    dims=["time"],
    name="temperature"
)

# Aggiungiamo metadati utilizzando `attrs`
temperature_da.attrs = {
    "units": "Celsius",
    "description": "Temperature measurements",
    "source": "Simulated data",
    "history": "Generated for a tutorial on xarray.attrs"
}

# Visualizziamo l'oggetto
print(temperature_da)

In [ ]:
#As for dictionaries:
print("Unità:", temperature_da.attrs["units"])

In [ ]:
temperature_da.attrs["source"] = "Real sensor data"
temperature_da.attrs["new_field"] = "Additional metadata"

In [ ]:
temperature_da.attrs

In [ ]:
del temperature_da.attrs["new_field"]

In [ ]:
temperature_da.attrs

A Dataset object can also contain metadata. You can add attributes both at the dataset level and to the individual DataArrays within it.

### Documentation

Xarray has two core data structures, which build upon and extend the core strengths of NumPy and pandas. Both data structures are fundamentally N-dimensional:

* DataArray is our implementation of a labeled, N-dimensional array. It is an N-D generalization of a pandas.Series
    
* Dataset is a multi-dimensional, in-memory array database. It is a dict-like container of DataArray objects aligned along any number of shared dimensions, and serves a similar purpose in xarray to the pandas.DataFrame.


# Open the netcdf file with xarray

In [ ]:
ds = xr.open_dataset('data/20230101_m-OGS--PFTC-MedBFM4-MED-b20230214_an-sv08.00_lev.nc')

#### Take a look at our dataset: 
The file is divided into 4 sections: Dimensions, Coordinates, Data variables, Attributes</br>
It is also possible to check the attributes and data representation

In [ ]:
ds

#### Inspect sections: two are dictionaries, two are arrays

In [ ]:
ds.dims

In [ ]:
ds.attrs

In [ ]:
ds.attrs['institution']

In [ ]:
ds.coords

In [ ]:
ds.data_vars

#### How can I access the data?

In [ ]:
ds.longitude

##### Different methods to access coordinates DataArrays

In [ ]:
print(ds.longitude)
print("%--------------------%")
print(ds["longitude"])
print("%--------------------%")
print(ds.coords["longitude"])

##### Use .values to access to the coordinates data values:

In [ ]:
print(ds.longitude.values)
print("%--------------------%")
print(ds["longitude"].values)
print("%--------------------%")
print(ds.coords["longitude"].values)

### Analyzing variables

##### To access the variables DataArray

In [ ]:
ds.phyc

In [ ]:
ds.chl

##### Reading attributes from DataArray

In [ ]:
print("Long name & standard name: ",ds.chl.long_name,ds.chl.standard_name,"\n")
print("%--------------------%")
print("Units: ",ds.chl.units,"\n")
print("%--------------------%")
print("Coordinates: ",ds["chl"].coords,"\n")
print("%--------------------%")
print("Attributes: ",ds["chl"].attrs,"\n")
print("%--------------------%")
print("Dimensions: ",ds.chl.dims,"\n")

In [ ]:
print(ds.chl.dims)
print("%--------------------%")
print(ds.chl.longitude.dims)

####  Define a new dataset containing only the variable chlorophyll and then starting from this dataset we try to extract subsets by exploiting the coordinates

| | Dimension lookup   | Index lookup    | DataArray syntax   |
|---:|:-------------|:-----------|:------|
| 1 | Positional  | By integer       | da[:, 0]   | 
| 2 | Positional  | By integer    | da.loc[:, 'IA']   |
| 3 | By name  | By integer    | da.isel(space=0)</br>or da[dict(space=0)]   |
| 4 | By name  | By label    | da.sel(space='IA')</br>or da.loc[dict(space='IA')]   |

In [ ]:
chlorophyll = ds.chl
chlorophyll

### Data selection and slicing

| | Dimension lookup   | Index lookup    | DataArray syntax   | Dataset syntax | Like what?
|---:|:-------------|:-----------|:------|:------------|:-------------|
| 1 | Positional  | By integer       | da[:, 0]   | not availabe | numpy |
| 2 | Positional  | By label    | da.loc[:, 'IA']   | not available | pandas |
| 3 | By name  | By integer    | da.isel(space=0)</br>or da[dict(space=0)]   | ds.isel(space=0)</br>or ds[dict(space=0)] | NA |
| 4 | By name  | By label    | da.sel(space='IA')</br>or da.loc[dict(space='IA')]   | da.sel(space='IA')</br>or ds.loc[dict(space='IA')] | NA |

#### Selecting by time

In [ ]:
chlorophyll.loc["2023-01-01"]

#### Selecting by a fixed depth

In [ ]:
chlorophyll[:, 10, :, :]

#### Slicing along the depth

In [ ]:
ds.isel(time=0, depth=slice(0, 2))

#### Select the coordinates closest to those given

In [ ]:
chlorophyll.sel(depth=[1, 30], method="nearest")

#### The following syntaxes do not work because it looks for coincident values with latitude equal to 34 or 38

In [ ]:
chlorophyll[0,0].where(chlorophyll.latitude.isin([34,38]))

In [ ]:
np.any(~np.isnan(chlorophyll[0,0].where(chlorophyll.latitude.isin([34,38])).values))

In [ ]:
chlorophyll[0,0].where((chlorophyll.latitude==34) & (chlorophyll.longitude==15))

In [ ]:
np.any(~np.isnan(chlorophyll[0,0].where((chlorophyll.latitude==34) & (chlorophyll.longitude==15)).values))

### Store the chlorophyll density at the surface level in the chlorophyll_0 data array 

In [ ]:
chlorophyll_0=chlorophyll[0, 0, :, :]#.drop_vars(['time','depth'])
chlorophyll_0

### Start plotting maps using the plotting function of xarray

In [ ]:
chlorophyll_0.plot()

### Changing colormap

In [ ]:
chlorophyll_0.plot(cmap='plasma')

### Plotting using coordinate slicing

In [ ]:
chlorophyll_0.where(chlorophyll_0.latitude < 42).plot()

In [ ]:
chlorophyll_0.where(chlorophyll_0.latitude +chlorophyll_0.longitude< 50).plot()

### Focusing on the sea of Sicily

In [ ]:
lon = ds.coords["longitude"]
lat = ds.coords["latitude"]
#chlorophyll[0,0].loc[dict(longitude=lon[(lon > 10) & (lon < 20)], latitude=lat[(lat > 36) & (lat < 40)])].plot()
chlorophyll_0.loc[dict(longitude=lon[(lon > 10) & (lon < 20)], latitude=lat[(lat > 36) & (lat < 40)])].plot()

#### In this case the slicing works because I am selecting latitude and longitude values corresponding to elements number 100 and 500 of the respective arrays</br>
Plot of  vertical profile of the chlorophyll density along depth

In [ ]:
chlorophyll.isel(latitude=126, longitude=115).plot()

### Trend of the chlorophyll density along the latitude

In [ ]:
chlorophyll[0, 1, :, 1]

In [ ]:
chlorophyll[0, 1, :, 10].plot.line(color="purple", marker="o")

### Plot of vertical profile and histogram of the chlorophyll density along depth using subplots

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(12,4))
chlorophyll.isel(latitude=126, longitude=115).plot(ax=axs[0])
chlorophyll.isel(latitude=126, longitude=115).plot.hist(ax=axs[1])
plt.tight_layout()
plt.draw()

### Compare the density of chlorophyll at different at different longitude

In [ ]:
chlorophyll.isel(depth=1, longitude=[15,20]).plot(x="latitude", hue="longitude")

### Some other plot available

In [ ]:
chlorophyll[0, 1, :, :].plot.contour()

In [ ]:
chlorophyll[0, 1, :, :].plot.contourf()

In [ ]:
chlorophyll[0, 1, :, :].plot.surface(color='red')